In [2]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
torch.cuda.is_available()

In [2]:
class RoadDataset(Dataset):
    def __init__(self, images_dir, masks_dir, image_transform=None, mask_transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.image_files = os.listdir(images_dir)
        self.mask_files = os.listdir(masks_dir)
        self.image_transform = image_transform
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        mask_path = os.path.join(self.masks_dir, self.mask_files[idx])

        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.image_transform:
            image = self.image_transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        mask = (mask > 0).float() 

        return image, mask


image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

mask_transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = RoadDataset("training/images", "training/groundtruth", image_transform=image_transform, mask_transform=mask_transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        def block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )

        self.enc1 = block(3, 64)
        self.enc2 = block(64, 128)
        self.enc3 = block(128, 256)
        self.enc4 = block(256, 512)

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = block(512, 1024)

        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = block(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = block(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = block(128, 64)

        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        bottleneck = self.bottleneck(self.pool(enc4))

        dec4 = self.dec4(torch.cat([self.up4(bottleneck), enc4], dim=1))
        dec3 = self.dec3(torch.cat([self.up3(dec4), enc3], dim=1))
        dec2 = self.dec2(torch.cat([self.up2(dec3), enc2], dim=1))
        dec1 = self.dec1(torch.cat([self.up1(dec2), enc1], dim=1))

        return self.final(dec1)

model = UNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [4]:
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(dataloader):.4f}")


torch.save(model.state_dict(), "road_detection_model.pth")

print("Training complete and model saved.")

Epoch 1/30, Loss: 0.4450
Epoch 2/30, Loss: 0.3483
Epoch 3/30, Loss: 0.2775
Epoch 4/30, Loss: 0.1798
Epoch 5/30, Loss: 0.1417
Epoch 6/30, Loss: 0.1279
Epoch 7/30, Loss: 0.1113
Epoch 8/30, Loss: 0.1024
Epoch 9/30, Loss: 0.0980
Epoch 10/30, Loss: 0.0880
Epoch 11/30, Loss: 0.0853
Epoch 12/30, Loss: 0.0734
Epoch 13/30, Loss: 0.0700
Epoch 14/30, Loss: 0.1010
Epoch 15/30, Loss: 0.0766
Epoch 16/30, Loss: 0.0588
Epoch 17/30, Loss: 0.0541
Epoch 18/30, Loss: 0.0537
Epoch 19/30, Loss: 0.0553
Epoch 20/30, Loss: 0.0609
Epoch 21/30, Loss: 0.0478
Epoch 22/30, Loss: 0.0449
Epoch 23/30, Loss: 0.0440
Epoch 24/30, Loss: 0.0435
Epoch 25/30, Loss: 0.0424
Epoch 26/30, Loss: 0.0420
Epoch 27/30, Loss: 0.0867
Epoch 28/30, Loss: 0.0434
Epoch 29/30, Loss: 0.0384
Epoch 30/30, Loss: 0.0370
Training complete and model saved.


In [9]:
model = UNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("road_detection_modelV1.pth", map_location=device))
model.to(device)
model.eval()

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def predict_image(model, image_path, transform, device, threshold=0.5):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        output = torch.sigmoid(output)
        output = (output > threshold).float()

    return output[0]

test_images_dir = "test/test"
output_masks_dir = "test/predicted_masks_test_v2"

os.makedirs(output_masks_dir, exist_ok=True)

test_image_files = os.listdir(test_images_dir)
for test_image_file in test_image_files:
    test_image_path = os.path.join(test_images_dir, test_image_file)

    predicted_mask = predict_image(model, test_image_path, test_transform, device)

    output_mask_path = os.path.join(output_masks_dir, test_image_file)
    save_image(predicted_mask, output_mask_path)

print(f"Predicted masks saved in {output_masks_dir}.")


Predicted masks saved in test/predicted_masks_test_v2.


In [1]:
source_dir = r"C:\Users\jakov\Desktop\hakatoon\test_set_images"
target_dir = r"C:\Users\jakov\Desktop\hakatoon\test\test"

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

image_extensions = {'.png'}

for root, dirs, files in os.walk(source_dir):
    for file in files:
        if os.path.splitext(file)[1].lower() in image_extensions:
            source_file_path = os.path.join(root, file)
            target_file_path = os.path.join(target_dir, file)
            
            if os.path.exists(target_file_path):
                base, ext = os.path.splitext(file)
                counter = 1
                while os.path.exists(target_file_path):
                    target_file_path = os.path.join(target_dir, f"{base}_{counter}{ext}")
                    counter += 1
            
            shutil.copy2(source_file_path, target_file_path)
            print(f"Copied: {source_file_path} -> {target_file_path}")

print("All images have been copied successfully.")

NameError: name 'os' is not defined